# Data Science Essentials: Data Cleaning
    Bryant McArthur
    Math 403
    October 11, 2022
    

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

### Problem 1

The g\_t\_results.csv file is a set of parent-reported scores on their child's Gifted and Talented tests. 
The two tests, OLSAT and NNAT, are used by NYC to determine if children are qualified for gifted programs.
The OLSAT Verbal has 16 questions for Kindergardeners and 30 questions for first, second, and third graders.
The NNAT has 48 questions. 
Using this dataset, answer the following questions.


1) What column has the highest number of null values and what percent of its values are null? Print the answer as a tuple with (column name, percentage). Make sure the second value is a percent.

2) List the columns that should be numeric that aren't. Print the answer as a tuple.

3) How many third graders have scores outside the valid range for the OLSAT Verbal Score? Print the answer

4) How many data values are missing (NaN)? Print the number.

Each part is one point.

In [2]:
# Read csv
df = pd.read_csv('g_t_results.csv')
print((df.isna().sum().idxmax(), df.isna().sum().max() / df['School Assigned'].size))

# get numeric columns
numeric_cols = ('OLSAT Verbal Score',
                 'OLSAT Verbal Percentile',
                 'NNAT Non Verbal Raw Score',
                )
print(numeric_cols)

# Rename
df = df.rename({'Entering Grade Level': 'Grade'}, axis=1)

# Third grade df
third_grade = df.query("Grade == '3'")
third_grade['OLSAT Verbal Score'] = third_grade['OLSAT Verbal Score'].to_numpy('float64')

# Filter
missing = (third_grade['OLSAT Verbal Score'] > 30) + (third_grade['OLSAT Verbal Score'] < 0)
print(missing.sum())
df.isna().sum().sum()

('School Assigned', 0.7521367521367521)
('OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score')


<ipython-input-2-6e3687cb5439>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  third_grade['OLSAT Verbal Score'] = third_grade['OLSAT Verbal Score'].to_numpy('float64')


1


C:\Users\bryan\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:203: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  warnings.warn(


192

### Problem 2

imdb.csv contains a small set of information about 99 movies. Clean the data set by doing the following in order: 

1) Remove duplicate rows by dropping the first **or** last. Print the shape of the dataframe after removing the rows.

2) Drop all rows that contain missing data. Print the shape of the dataframe after removing the rows.

3) Remove rows that have data outside valid data ranges and explain briefly how you determined your ranges for each column.

4) Identify and drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.

5) Convert the titles to all lower case.

Print the first five rows of your dataframe.

In [5]:
# Read csv
imdb = pd.read_csv('imdb.csv')

# Filter duplicates and NaNs
imdb.drop_duplicates(keep='first', inplace=True)
print(imdb.shape)
imdb.dropna(inplace=True)
print(imdb.shape)

# Make Numpy
imdb = imdb.replace(('Nan', 'Null'), np.nan)
imdb = imdb[(imdb.imdb_score > 0)]
imdb = imdb[(imdb.duration < 200)]
imdb = imdb[(imdb.duration > 0)]
imdb.dropna(inplace=True)

# Drop cols
drop_cols = list()
for column in imdb.columns:
    if len(imdb[column].unique()) <= 3:
        drop_cols.append(column)

print(tuple(drop_cols))
imdb.drop(drop_cols, axis=1, inplace=True)

# Lowercase titles
imdb['movie_title'] = imdb['movie_title'].str.lower()
imdb.head(5)

(93, 13)
(64, 13)
('color', 'language')


,director_name,duration,gross,genres,movie_title,title_year,country,budget,imdb_score,actors,movie_facebook_likes
1,Shane Black,195,408992272.0,Action|Adventure|Sci-Fi,iron man 3,2013,USA,200000000.0,7.2,"Robert Downey Jr.,Jon Favreau,Don Cheadle",95000
2,Quentin Tarantino,187,54116191.0,Crime|Drama|Mystery|Thriller|Western,the hateful eight,2015,USA,44000000.0,7.9,"Craig Stark,Jennifer Jason Leigh,Zoë Bell",114000
3,Kenneth Lonergan,186,46495.0,Drama,margaret,2011,usa,14000000.0,6.5,"Matt Damon,Kieran Culkin,John Gallagher Jr.",0
4,Peter Jackson,186,258355354.0,Adventure|Fantasy,the hobbit: the desolation of smaug,2013,USA,225000000.0,7.9,"Aidan Turner,Adam Brown,James Nesbitt",83000
8,Joss Whedon,173,623279547.0,Action|Adventure|Sci-Fi,the avengers,2012,USA,220000000.0,8.1,"Chris Hemsworth,Robert Downey Jr.,Scarlett Joh...",123000


### Problem 3

basketball.csv contains data for all NBA players between 2001 and 2018.
Each row represents a player's stats for a year.

Create two new features:

    career_length (int): number of years player has been playing (start at 0).
    
    target (str): The target team if the player is leaving. If the player is retiring, the target should be 'retires'.
                  A player is retiring if their name doesn't exist the next year.
                  (Set the players in 2019 to NaN).

Remove all duplicate players in each year.
Remove all rows except those where a player changes team, that is, target is not null nor 'retires'.

Drop the player, year, and team_id columns.

Return the first 10 lines of your dataframe.

In [6]:
# Read csv make career length
df = pd.read_csv('basketball.csv')
df['career length'] = df.apply(lambda x: x['year'] - df.loc[bb['player'] == x['player'],['year']].min(), axis=1)

# Drop duplicates and make target col
df.drop_duplicates(subset=['player','year'], inplace=True)
df['target'] = df.groupby(['player'])['team_id'].shift(1)

# Account for retiring players
for row in df[df['target'].isna()].index:
    if df.loc[row,'year'] != 2019:
        df.loc[row, 'target'] == 'retires'
        
# Iterate through rows in bb index
for row in bb.index:
    if df.loc[row, 'team_id'] == df.loc[row, 'target']:
        df.drop(row, inplace=True)
        
# Clean Nans
df.dropna(inplace=True)
df.drop(['player','year','team_id'], axis=1, inplace=True)
df.head(10)

,age,per,ws,bpm,career length,target
453,27,8.2,1.0,-2.5,5,PHO
461,24,13.0,1.2,-0.9,2,ATL
462,24,15.9,6.2,2.9,3,MEM
464,33,12.7,3.7,-1.9,14,HOU
467,32,11.8,5.3,0.7,13,PHO
477,29,7.5,1.1,-2.8,9,MIN
482,31,14.1,1.9,-0.2,10,SAS
489,25,14.1,2.9,-2.4,6,CHO
490,29,12.6,2.8,0.1,2,SAC
493,28,13.0,0.0,-3.2,7,MIL


### Problem 4

Load housing.csv into a dataframe with index=0. Descriptions of the features are in housing_data_description.txt.  
The goal is to construct a regression model that predicts SalePrice using the other features of the dataset.  Do this as follows:

	1) Identify and handle the missing data.  Hint: Dropping every row with some missing data is not a good choice because it gives you an empty dataframe.  What can you do instead?
    FIXME
	2) Identify the variable with nonnumeric values that are misencoded as numbers.  One-hot encode it. Hint: don't forget to remove one of the encoded columns to prevent collinearity with the constant column (which you will add later).
    
    3) Add a constant column to the dataframe.

    4) Save a copy of the dataframe.

	5) Choose four categorical featrues that seem very important in predicting SalePrice. One-hot encode these features and remove all other categorical features.
		
	6) Run an OLS using all numerical data regression on your model.  

	
Print the ten features that have the highest coef in your model and the summary. Don't print the OLS

In [9]:
df = pd.read_csv('housing.csv', index_col=0)

# find nan columns and drop
nan_cols = list()
for col in df.columns:
    if pd.isna(df[col].value_counts(dropna=False).idxmax()):
        nan_cols.append(col)
    
    else:
        # Fill Nan with mode
        df[col].fillna(df[col].mode()[0], inplace=True)
        
df = df.drop(columns=nan_cols)

# One-hot categorize
df = pd.get_dummies(df, columns=['MSSubClass'], drop_first=True)

# Add constant column
df['const'] = np.ones(df.shape[0])

# make a copy
new_df = df.copy()

In [10]:
# keep the important cols and price
important_cols = ['SalePrice','HouseStyle','KitchenQual','OverallCond','OverallQual']
new_df = new_df.loc[:, important_cols]
new_df = pd.get_dummies(new_df, columns=important_cols[1:], drop_first=True)
new_df['const'] = np.ones(new_df.shape[0])

# run regression
y = new_df['SalePrice']
X = new_df.iloc[:,1:]
reg = sm.OLS(y, X).fit()

# save as html then as pd.DataFrame
a = reg.summary()
reg_as_html = a.tables[1].as_html()
reg_df = pd.read_html(reg_as_html, header=0, index_col=0)[0]

# sorts and prints top 10 highest features
reg_df = reg_df.sort_values(by='coef', ascending=False).reset_index()
print(reg_df['index'].iloc[:10])

0       OverallQual_10
1        OverallQual_9
2        OverallQual_8
3                const
4        OverallQual_7
5        OverallQual_6
6        OverallQual_5
7        OverallQual_4
8    HouseStyle_2.5Fin
9        OverallQual_3
Name: index, dtype: object


In [11]:
print(a)

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.711
Model:                            OLS   Adj. R-squared:                  0.706
Method:                 Least Squares   F-statistic:                     130.5
Date:                Tue, 11 Oct 2022   Prob (F-statistic):               0.00
Time:                        15:23:13   Log-Likelihood:                -17638.
No. Observations:                1460   AIC:                         3.533e+04
Df Residuals:                    1432   BIC:                         3.548e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
HouseStyle_1.5Unf -3.564e+04   1.21e+0

### Problem 5

Using the copy of the dataframe you created in Problem 4, one-hot encode all the categorical variables.
Print the shape of the dataframe and run OLS.

Print the ten features that have the highest coef in your model and the summary.
Write a couple of sentences discussing which model is better and why.

In [12]:
df = pd.get_dummies(df, drop_first=True)
print(df.shape)

# Get list of columns we want
cols = list(df.columns)
cols.remove('SalePrice')
y = df['SalePrice']
X = df.loc[:, cols]

# Run regression
results = sm.OLS(y, X).fit()

# Make html and df
summary_results = results.summary()
results_as_html = summary_results.tables[1].as_html()
results_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

# sorts and prints
results_df = results_df.sort_values(by='coef', ascending=False).reset_index()
print(results_df['index'].iloc[:10])

(1460, 245)
0    RoofMatl_Membran
1      RoofMatl_Metal
2    RoofMatl_WdShngl
3    RoofMatl_Tar&Grv
4    RoofMatl_CompShg
5       RoofMatl_Roll
6    RoofMatl_WdShake
7       GarageCond_TA
8       GarageCond_Gd
9       GarageCond_Po
Name: index, dtype: object


It is obviously way more helpful to clean the data before using it and running a regression. Filtering for variables that matter to what you want make way more sense than running it with everything. 